# Coordinate Calculator and Label Compiler

Inputs:
- location in pixels of beginning and end of axes
- range of axes
- units of axis range
- user's desired spacing for x coordinates
- NO legend input! how to specify different colors/markers that separates the curves (for later stages)
    - could eventually have mask look for the legend (distinctive enough) to learn what types of distinguishing features exist between the lines on the graph
    - I will ignore this part for now
- detectron2 output:
    - list of dicts, one dict for each image
    - each dict has an object 'instances' with the following fields.
        - pred_masks: a tensor of shape (N,H,W) for N instances and an image of height H and width W.
        - access it using: ```outputs['instances'].pred_masks```
        - True False for each pixel
    
Outputs:
- dictionary with the following fields:
    - coordinates: a dictionary, with fields equal to the curve labels (one for each label), and entries that are a list of tuples, each with (x, and y coordinates in same units as input image)
    - units: (a tuple of x and y units)
    - beginning and end of curve
    - NO labels (for later stages): a list of labels for each curve, taken from legend if possible, or automatic (data1, data2, etc)


Options for coordinates:
1. based on the actual axis, choose a set of x coordinates for which we want to know the y coordinates (for each curve)
2. based on pixels, grab points that occur every "few" pixels along the x axis, then convert to coordinates

- could do the first option by finding out how many pixels are between each of the given x points, and using that number of pixels paired with the second option.
- do we want user to input the spacing between points, or should we use a default value that seems reasonable based on the range of the x axis?
- should the different curves have uniform x axis values (so y values are directly comparable), or should we have the sets of data points for each curve start at the 'beginning' of the curve and end at the 'end'


Outlier cases:
- nested plots
- weird legend formatting

uniform x axis but also save the very first and last endpoints of the curve 

maybe incorporate later:

convert ID to the label from the legend input

def get_label_for_id():
    return label

### Notes, TODO
- adjust the x and y pixel locations to lie on the same horizontal and vertical lines
- put the coordinate and pixel min max values in a dictionary or something more compact
- use os to create a directory for each project. currently puts excel files in the current folder

In [1]:
import pandas as pd
import numpy as np

This is the first function called in my code. The arguments are all of the user input that I need

In [67]:
# create the axis info dictionary
# note: ypixelmin < ypixelmax
def get_axis_info(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax, ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax, max_points, units):
    # if the input from kev is in a more compact form extract this info below before passing into the scale funcitons
    x_scale = get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax)
    y_scale = get_x_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax)
    pixel_origin = (xpixelmin, ypixelmax) # assumes that y pixel min is the larger y value
    axis_info_dict = {'pixel_origin': pixel_origin,
                      'x_scale': x_scale,
                      'y_scale': y_scale,
                      'step': get_step(max_points, xpixelmin, xpixelmax),
                      'units': units,
                      'y_pixel_range': (ypixelmin, ypixelmax),
                      'x_pixel_range': (xpixelmin, xpixelmax)}
    return axis_info_dict

In [23]:
def test_get_axis_info():
    # the output is a dictionary with the fields: pixel_origin, x_scale, y_scale, step, and units
    axis_info_dict = get_axis_info(1, 5, 20, 250, 10, 25, 30, 280, 30, ('volts', 'amps'))
    assert isinstance(axis_info_dict, dict), 'axis_info_dict is not a dictionary'
    for field in ['step', 'pixel_origin', 'x_scale', 'y_scale', 'units', 'y_pixel_range', 'x_pixel_range']:
        assert field in axis_info_dict.keys(), 'axis_info_dict is missing fields'
    return
test_get_axis_info()

In [32]:
# convert the max points desired into a step size in pixels
def get_step(max_points, xpixelmin, xpixelmax):
    #step = length of x axis in pixels / max_points
    step = np.ceil((xpixelmax - xpixelmin) / max_points)
    return int(step)

In [36]:
def test_get_step():
    step1 = get_step(19, 10, 200)
    step2 = get_step(18, 10, 200)
    step3 = get_step(16, 10, 200)
    # the step size * the number of points should be close to the length of the axis
    # step size is an integer
    for step in [step1, step2, step3]:
        assert isinstance(step, int), 'the step size is not an integer'
    # the length of the axis/ step size should be close to but less than the max points
    assert np.isclose(190 / step1, 19), 'length of axis/step size not ~< max points'
    assert ((190 / step2) < 18) and ((190 / step2) > 17), 'length of axis/step size not ~< max points'
    assert ((190 / step3) < 16) and ((190 / step3) > 15), 'length of axis/step size not ~< max points'
    return
test_get_step()

In [42]:
# establish scaling from pixel to real units
def get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax):
    # the x pixel and x coordinate count up in the same direction
    pixel_range = xpixelmax - xpixelmin
    coordinate_range = xcoordinatemax - xcoordinatemin
    x_scale = pixel_range / coordinate_range
    return x_scale # pixels per coordinate

#if the ypixelmin is the smaller of the ypixel values, then this is the same as the x scaling. just using get_scale for now
def get_y_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax):
    # y pixel count down and y coordinate count up from origin
    pixel_range = ypixelmin - ypixelmax
    coordinate_range = ycoordinatemax - ycoordinatemin
    y_scale = pixel_range / coordinate_range
    return y_scale # pixels per coordinate

In [46]:
def test_get_x_scale():
    x_scale = get_x_scale(1, 5, 20, 250)
    # x_scale * coordinate range should equal pixel range
    assert np.isclose(x_scale * (5-1), (250-20)), 'the x scaling is incorrect'
    assert np.isclose(x_scale, 57.5), 'the x scaling is incorrect'
    
    x_scale = get_x_scale(-1, -5, 20, 250)
    assert np.isclose(x_scale * (-5+1), (250-20)), 'the x scaling is incorrect'
    assert np.isclose(x_scale, -57.5), 'the x scaling is incorrect'
    return
test_get_x_scale()

def test_get_y_scale():
    y_scale = get_y_scale(10, 25, 30, 280)
    # y_scale should be greater than zero
    assert y_scale > 0, 'y scale should be greater than zero'
    # y_scale * coordinate range should equal pixel range
    assert np.isclose(y_scale * (25-10), (280-30)), 'the y scaling is incorrect'
    assert np.isclose(y_scale, 16.666666666), 'example y scale is incorrect'
    return
test_get_y_scale()

In [47]:
# convert pixel location to coordinates
def pixel_to_coords(pixel_loc, axis_info_dict):
    # pixel_loc is a tuple (x,y) of pixel location starting from top left
    x_pixel_loc = pixel_loc[0]
    coord_x = x_pixel_to_coords(x_pixel_loc, axis_info_dict)
    
    # get signed distance from pixel to origin in x and y(pixel units):
    pixel_distance_y = axis_info_dict['pixel_origin'][1] - pixel_loc[1]
    
    # pixels / (pixel/coord) = coord
    coord_y = pixel_distance_y / axis_info_dict['y_scale']
    return (coord_x, coord_y)

In [48]:
# for use later in code, just the x axis pixel to coords
def x_pixel_to_coords(x_pixel_loc, axis_info_dict):
    pixel_distance_x = x_pixel_loc - axis_info_dict['pixel_origin'][0]
    coord_x = pixel_distance_x / axis_info_dict['x_scale']
    return coord_x

In [54]:
def test_pixel_to_coords():
    axis_info_dict1 = {'pixel_origin': (20, 100), 'y_scale': 5.3, 'x_scale': 20.5}
    axis_info_dict2 = {'pixel_origin': (20, 100), 'y_scale': -0.2, 'x_scale': 0.005}
    # the output coordinates should be within the coordinate ranges for each axis
    # given a scale and a location, test a few cases (+-0)
    coords1 = pixel_to_coords((20, 100), axis_info_dict1) # (0,0)
    coords2 = pixel_to_coords((20, 100), axis_info_dict2) # (0,0)
    coords3 = pixel_to_coords((55, 33), axis_info_dict1) # (1.707317, 12.641509)
    coords4 = pixel_to_coords((55, 33), axis_info_dict2) # (7000, -335)
    coords5 = pixel_to_coords((55, 105), axis_info_dict2) # (1.707317, 25)
    
    assert np.isclose(coords1[0], 0), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords1[1], 0), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords2[0], 0), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords2[1], 0), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords3[0], 1.707317), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords3[1], 12.64150943), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords4[0], 7000), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords4[1], -335), 'pixel to coordinate conversion is incorrect'
    assert np.isclose(coords5[1], 25), 'pixel to coordinate conversion is incorrect'
    return
test_pixel_to_coords()

#dont really need this. accomplished in the unify x function
#pretty much get the avg y pixel function for each unique x pixel value
def clean_pixel_lst(pixel_lst):
    return cleaned_pixel_lst

In [10]:
# find closest number in list to value val
def closest(lst, val):
    lst = np.asarray(lst)
    idx = (np.abs(lst - val)).argmin()
    return lst[idx]

In [59]:
def test_closest():
    lst = [0, 2, 1, 3, 4, 5, 6]
    # val is equidistant to two values in list, first one in list is chosen
    assert closest(lst, 1.5) == 2, 'closest value is incorrect'
    assert closest(lst, 3.5) == 3, 'closest value is incorrect'
    # val is equal to one value in list
    assert closest(lst, 2) == 2, 'closest value is incorrect'
    # val is closer to one in particular
    assert closest(lst, 1.8) == 2, 'closest value is incorrect'
    return
test_closest()

In [106]:
# return x and y values of the unified list
def unify_x(pixel_lst, axis_info_dict):
    # get the number of pixels between each desired coordinate pt based on scale
    # create bins of pixels and avg the values between them
    pixel_lst.sort()
    
    # step is a global variable based on user input
    step = axis_info_dict['step']
    x_pixels = [i[0] for i in pixel_lst]
    x_vals = list(range(min(x_pixels), max(x_pixels), step))

    
    # create dictionary of point and closest standard x val
    closest_dict = {}
    for point in pixel_lst:
        key = closest(x_vals, point[0])
        if key in closest_dict.keys():
            closest_dict[key].append(point)
        else:
            closest_dict[key] = [point]
    
    # iterate through keys to average all y values in each set
    for key in closest_dict:
        y_vals = [i[1] for i in closest_dict[key]]
        y_val = sum(y_vals) / len(y_vals)
        closest_dict[key] = y_val
    
    # for all the missing dict keys, make a line between nearest values and fill it in
    existing_keys = list(closest_dict.keys())
    existing_keys.sort()
    for x in x_vals:
        if x not in existing_keys:
            # find the index of first existing x greater than x
            i = 0
            while existing_keys[i] < x and i < (len(x_vals) + 1):
                i += 1
            
            x2 = existing_keys[i] # existing x just above missing x
            y2 = closest_dict[x2]
            x1 = existing_keys[i-1] # existing x just below missing x
            y1 = closest_dict[x1]
            
            # find line between bounds
            m = (y1 - y2) / (x1 - x2)
            b = (x1 * y2 - x2 * y1) / (x1 - x2)
            
            # solve for y of x
            y = m * x + b
            closest_dict[x] = y

        else:
            continue
    
    # turn dictionary into a list of tuples
    unified_pixel_lst = list(closest_dict.items())
    unified_pixel_lst.sort()
    
    
    return unified_pixel_lst # a list of (x, y) tuples

In [107]:
def test_unify_x():
    axis_info_dict = {'step': 3}
    pixel_lst = [(20, 100), (20, 90), (21, 91), (22, 85), (22, 83), (23, 80), (24, 81), (24, 83), (25, 80), (29, 50), (29, 45), (30, 30), (30, 10)]
    pixels_y = [i[1] for i in pixel_lst]
    pixels_x = [i[0] for i in pixel_lst]
    unified_pixel_lst = unify_x(pixel_lst, axis_info_dict)
    unified_x = [i[0] for i in unified_pixel_lst]
    unified_y = [i[1] for i in unified_pixel_lst]
    x_spaces = np.diff(unified_x)
    # the x values in the list of tuples are all stepsize apart
    assert np.allclose(x_spaces, 3), 'the spacing between x values is incorrect'
    # the x values are unique
    assert len(set(unified_x)) == len(unified_x), 'the x values are not unique'
    # y values are all between the min and max pixel
    for y in unified_y:
        assert y <= max(pixels_y) and y >= min(pixels_y), 'unified y value is outside of expected bounds'
    # same for x values
    for x in unified_x:
        assert x <= max(pixels_x) and x >= min(pixels_x), 'unified x value is outside of expected bounds'
    return
test_unify_x()

In [90]:
# modified


# looks at one N and all H,W values in that 'slice'
# each curve is one ID. get all pixel locations for one ID
def get_pixels_2d(pixel_array_2d):
    result = np.where(pixel_array_2d == True)
    pixel_lst = list(zip(result[1], result[0])) # gives the x then the y in the tuple
    return pixel_lst

In [94]:
def test_get_pixels_2d():
    # create an array with a few trues and many falses, check positions in output match
    test_array = np.array([[ False,  True, False,  False, False, False, False],
       [ False, False,  False,  True, False, False, False],
       [ False,  False, False, False, False,  True, False],
       [True, False, False,  False, False, False, True],
       [False, False,  False, False, False, False, False]])
    pixel_lst = get_pixels_2d(test_array)
    expected_pixel_lst = [(1, 0), (3, 1), (5, 2), (0, 3), (6, 3)]
    assert set(expected_pixel_lst) == set(pixel_lst), 'unexpected values in 2d pixel list'
    return
test_get_pixels_2d()

In [114]:
# testing
zeros = np.random.rand(2,3,4)
np.random.choice([True, False], (2, 5, 7), p=[0.2, 0.8]);

In [100]:
# modified

# ID = curve_N, N starts at 1 not zero
# step 1 to create coordinate dictionary to add to the output dict
def create_pixel_dict(pred_masks):
    # initialize dict
    pixel_dict = {}
    
    pixel_array_3d = np.array(pred_masks)
    for N in range(len(pixel_array_3d)):
        pixel_array_2d = pixel_array_3d[N]
        pixel_lst = get_pixels_2d(pixel_array_2d)
        
        # add the list of pixels for this N to the pixel dict
        ID = 'curve_' + str(N+1)
        pixel_dict[ID] = pixel_lst
    return pixel_dict

In [104]:
def test_create_pixel_dict():
    pred_masks = np.array([[[False,  True, False,  True, False, False, False],
        [False, False, False, False, False,  True,  True],
        [False,  True, False, False, False, False, False],
        [False,  True,  True, False, False, False, False],
        [False, False,  True, False, False, False, False]],

       [[ True, False, False, False, False,  True,  True],
        [ True, False, False, False,  True, False, False],
        [False, False, False, False, False,  True, False],
        [False, False,  True, False,  True, False, False],
        [False, False, False, False,  True, False, False]]])
    pixel_dict = create_pixel_dict(pred_masks)
    # check that the keys are 'curve_' and then a unique number that matches the shape of pred_masks
    assert list(pixel_dict.keys()) == ['curve_1', 'curve_2'], 'incorrect keys in pixel_dict'
    # the values are not the same for each one (not repeated)
    assert set(pixel_dict['curve_1']) != set(pixel_dict['curve_2']), 'curves in pixel_dict are not unique'
    return
test_create_pixel_dict()

In [14]:
# create coordinate dictionary to add to the output dict
def create_coordinate_dict(pixel_dict, axis_info_dict):
    # initialize dict
    coordinate_dict = {}
    
    for ID in pixel_dict.keys():
        pixel_lst = pixel_dict[ID]
        
        # get unified x axis:
        # add an if statement to handle user specifiying either step size (in coordinates) or number of points
        unified_pixel_lst = unify_x(pixel_lst, axis_info_dict)
        
        # convert pixels to coordinates
        coordinate_lst = []
        for pixel_loc in unified_pixel_lst:
            coordinate_lst.append(pixel_to_coords(pixel_loc, axis_info_dict))
        
        # add the list of coordinates for this ID to the coordinate dict
        coordinate_dict[str(ID)] = coordinate_lst
    return coordinate_dict

In [108]:
def test_create_coordinate_dict():
    pred_masks = np.array([[[False,  True, False,  True, False, False, False],
        [False, False, False, False, False,  True,  True],
        [False,  True, False, False, False, False, False],
        [False,  True,  True, False, False, False, False],
        [False, False,  True, False, False, False, False]],

       [[ True, False, False, False, False,  True,  True],
        [ True, False, False, False,  True, False, False],
        [False, False, False, False, False,  True, False],
        [False, False,  True, False,  True, False, False],
        [False, False, False, False,  True, False, False]]])
    pixel_dict = create_pixel_dict(pred_masks)
    axis_info_dict = {'pixel_origin': (0, 4), 'y_scale': 5.3, 'x_scale': 20.5, 'step': 1}
    coordinate_dict = create_coordinate_dict(pixel_dict, axis_info_dict)
    # the keys of the coordinate dict are the same as that of the pixel dict
    assert set(coordinate_dict.keys()) == set(pixel_dict.keys()), 'keys of coordinate and pixel dicts dont match'
    return
test_create_coordinate_dict()

In [110]:
# get the coordinate locations of the start and end of the curve
def get_start_end(pixel_dict, axis_info_dict):
    # initialize dict
    start_end_dict = {}
    
    for ID in pixel_dict.keys():
        pixel_lst = pixel_dict[ID]
        
        # get start and end, assumes x is in the first position
        start = x_pixel_to_coords(min(pixel_lst)[0], axis_info_dict)
        end = x_pixel_to_coords(max(pixel_lst)[0], axis_info_dict)
        # add the start and end tuple for this ID to the pixel dict
        start_end_dict[str(ID)] = (start, end)
    return start_end_dict

In [112]:
def test_get_start_end():
    pred_masks = np.array([[[False,  True, False,  True, False, False, False],
        [False, False, False, False, False,  True,  True],
        [False,  True, False, False, False, False, False],
        [False,  True,  True, False, False, False, False],
        [False, False,  True, False, False, False, False]],

       [[ True, False, False, False, False,  True,  True],
        [ True, False, False, False,  True, False, False],
        [False, False, False, False, False,  True, False],
        [False, False,  True, False,  True, False, False],
        [False, False, False, False,  True, False, False]]])
    pixel_dict = create_pixel_dict(pred_masks)
    axis_info_dict = {'pixel_origin': (0, 4), 'y_scale': 5.3, 'x_scale': 20.5, 'step': 1}
    start_end_dict = get_start_end(pixel_dict, axis_info_dict)
    # the keys of the output dict are the same as the input
    assert set(start_end_dict.keys()) == set(pixel_dict.keys()), 'keys of start_end and pixel dicts dont match'
    for key in start_end_dict.keys():
        # the end is larger than (or equal to) the start
        assert start_end_dict[key][1] >= start_end_dict[key][0], 'starting x value greater than end value'
        # the start and end are between the coordinate ranges (hard to test without using all other functions)
    return
test_get_start_end()

In [17]:
# modified



# create the output dictionary and input the units of axis and labels directly from the input info,
# as well as the coordinate dictionary
def create_output_dict(pred_masks, axis_info_dict):
    pixel_dict = create_pixel_dict(pred_masks)
    output_dict = {}
    output_dict['coordinates'] = create_coordinate_dict(pixel_dict, axis_info_dict)
    output_dict['start_end'] = get_start_end(pixel_dict, axis_info_dict)
    output_dict['units'] = axis_info_dict['units']
    return output_dict

In [19]:
# saves the resulting coordinates and start/stop info in an excel file with one sheet per curve
def write_results_to_excel(output_dict, filename):
    excel_filename = str(filename) + '.xlsx'
    writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')
    x_units = output_dict['units'][0]
    y_units = output_dict['units'][1]
    
    # a summary of the start and end of the x axis for each ID
    starts = []
    ends = []
    ids = []
    for ID in output_dict[start_end].keys():
        ids.append(ID)
        start = output_dict['start_end'][ID][0]
        end = output_dict['start_end'][ID][1]
        starts.append(start)
        ends.append(end)
    df = pd.DataFrame([ids, starts, ends],  columns=['ID', 'x start '+str(x_units), 'x end '+str(x_units)])
    df.to_excel(writer, sheet_name='starts_ends')
    
    # the actual data in xy form, one ID per sheet
    for ID in output_dict['coordinates'].keys():
        x = output_dict['coordinates'][ID][0]
        y = output_dict['coordinates'][ID][1]
        column_titles = ['x, ' + str(x_units), 'y, ' + str(y_units)]
        df = pd.DataFrame([x, y], columns=column_titles)
        df.to_excel(writer, sheet_name=str(ID))
    writer.save()

In [20]:
# modified - might need to check if there are more than one image before indexing for multiple


# loops through each image in the output of detectron2
def datayoink_to_csv(detect_output, axis_info_dict):
    n_images = len(detect_output)
    for image in range(n_images):
        pred_masks = detect_output[image]['instances'].pred_masks
        output_dict = create_output_dict(pred_masks, axis_info_dict)
        filename = 'image_' + str(image)
        write_results_to_excel(output_dict, filename)
    return

In order to use these functions, code looks like this:

In [ ]:
# detect_output is the output of detectron2 - list of instances objects
axis_info_dict = get_axis_info(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax, ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax, max_points, units)
datayoink_to_csv(detect_output, axis_info_dict)